## Урок 7. Нестандартные ситуации

### На основе выбранных продуктов из первого вебинара, выберите 2 A/B-теста и составьте план эксперимента, чтобы избежать Peeking Problem 
### Распишите, какие метрики и как вы будете анализировать

Peeking Problem - (проблема подглядывания) проявляется, когда вы проверяете промежуточные результаты с готовностью принять решение: раскатить одну из версий, если различие между тестом и контролем окажется значимым .
Это может привести к двум типам последствий:
1) Преждевременно законченный тест - не валидные результаты.  
2) Продолжение теста в ожидании стат значимости - шанс потратить деньги и время компании не получив результата.

Выбор продукта - wildberries.ru.

**А/В тесты:**  
1. Если каждому клиенту предоставлять накопительные баллы в зависимости от суммы покупок, то Churn-rate уменьшится, а LTV повысится.  
2. Если продавцы будут предоставлять более качественные фото товаров, то Churn-rate также уменьшится, а CR увеличится.

**Ключевые метрики:**  

1. LTV - доход, который клиент принёс компании за всё время взаимодействия с ней;  
2. AOV - средний чек;  
3. CR - конверсия в покупку;  
4. Churn-rate - количество отказов.

In [1]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

In [2]:
def calc_continuous_effect_size(mean_1: Union[float, int],
                                std_1: Union[float, int],
                                mean_2: Union[float, int],
                                std_2: Union[float, int],
                                nobs_1: int = 10_000,
                                nobs_2: int = 10_000):
    
    es_formula = (mean_1 - mean_2) / ((std_1**2 + std_2**2) / 2) ** 0.5
    es_import = effectsize_smd(mean_1, std_1, nobs_1, mean_2, std_2, nobs_2)[0]
    
    return es_formula, es_import 

In [3]:
def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

In [4]:
mu_control, mu_test = 4.2, 4.83
std_control, std_test = 35, 35 #* 1.15

calc_continuous_effect_size(mean_1=mu_control, std_1=std_control, mean_2=mu_test, std_2=std_test)

(-0.018, -0.017999324924053955)

In [5]:
effect_size = calc_continuous_effect_size(mean_1=mu_control, std_1=std_control, mean_2=mu_test, std_2=std_test)[1]
selection = calc_sample_size_continuous(effect_size=effect_size, alpha=.05, beta=.2)

print(f"Общий размер выборки {selection} человек.")

Общий размер выборки 96908 человек.


In [6]:
days = selection/20000

print(f"Необходимо провести эксперимент в течение {days} дней")

Необходимо провести эксперимент в течение 4.8454 дней


Данные 4.8454 дней округляем в большую сторону ~ 5 дней.

### Шаблон дизайна эксперимента №1.

**Гипотеза:**  
1. Если каждому клиенту предоставлять накопительные баллы 10% в зависимости от суммы покупок, то AOV=4,2 т.р повысится на 15%, потому что клиенты будут стараться накопить баллы, и будут совершать больше покупок.  
2. А Churn-rate (отказы) уменьшится на 10%, потому что клиенты будут стараться накопить баллы, и реализовать их, более тщательно выбирать товары.  
  
**Что делаем:**
1. Контрольная версия: контрольной группе не начисляют накопительные баллы.  
2. Тестовая версия: тестовой группе начисляют 10% баллов от покупки.  
  
**На каких пользователях тестируем:**  
* Тестируем на 20 000 клиентах маркетплейса wildberries.ru, 2 группы по 10 000 клиентов в каждой.  
  
**Метрики:**  
1. LTV - доход, который клиент принёс компании за всё время взаимодействия с ней.  
2. Churn-rate - количество отказов.  
  
**План действий:**  
1. Если наш эксперимент будет положительным и мы зафиксируем ожидаемое увеличение LTV и уменьшение Churn-rate, то масштабируем изменение и «выкатываем» его на всех пользователей.  
2. Если LTV падает и/или Churn-rate растет, откатываем эксперимент.  
  
**Избежать Peeking Problem:**  
1. Правильно рассчитываем длительность теста и придерживаться сроков, которые вы посчитали.  
2. Мониторить промежуточные результаты эксперимента можно и нужно, но принимать решения на основе неполных данных нельзя.

### Шаблон дизайна эксперимента №2.

**Гипотеза:**  
1. Если продавцы будут предоставлять более качественные детализированные фото товаров, то CR увеличится на 5%, потому что клиенты могут рассмотреть фото товара в деталях, и выбрать нужный товар, а не пролистнуть дальше.  
2. Churn-rate уменьшится на 10%, потому что яркие, светлые, качественные фото товара, привлекают и стимулируют клиента к покупке.  
  
**Что делаем:**  
1. Контрольная версия: контрольной группе оставляем фото в карточках как было.  
2. Тестовая версия: тестовой группе показываем карточки с более качественными и яркими фото.  
  
**На каких пользователях тестируем:**  
* Тестируем на 20 000 клиентах маркетплейса wildberries.ru, 2 группы по 10 000 клиентов в каждой.  
  
**Метрики:**  
1. CR - конверсия в покупку;  
2. Churn-rate - количество отказов.  
  
**План действий:**  
1. Если наш эксперимент будет положительным и мы зафиксируем ожидаемое увеличение CR и уменьшение Churn-rate, то масштабируем изменение и «выкатываем» его на всех пользователей.  
2. Если CR падает и/или Churn-rate растет, откатываем эксперимент.  
  
**Избежать Peeking Problem:**  
1. Правильно рассчитываем длительность теста и придерживаться сроков, которые вы посчитали.  
2. Мониторить промежуточные результаты эксперимента можно и нужно, но принимать решения на основе неполных данных нельзя.